In [2]:
import os
import json
import pandas as pd
import folium
import gpxpy
import gpxpy.gpx
import requests
import math

### Helper functions

In [3]:
# Read and combine all json files
def read_json_files(folder_path):
    combined_edges = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                edges = data["data"]["ntb_findCabins"]["edges"]
                combined_edges.extend(edges)
    return combined_edges


In [4]:
# Convert data into pandas dataframe
def convert_to_dataframe(data):
    records = []
    for item in data:
        node = item["node"]
        geometry = node["geometry"]
        record = {
            "id": node["id"],
            "name": node["name"],
            "serviceLevel": node["serviceLevel"],
            "dntCabin": node["dntCabin"],
            "ownername": node["owner"]["name"],
            "latitude": geometry["coordinates"][1],  # longitude
            "longitude": geometry["coordinates"][0],  # latitude
            "height": geometry["coordinates"][2],  # height
            "areaName": node["areas"][0]["name"] if node["areas"] else "",  # area name
            "dntKey": node["openingHours"][0]["key"] if node["openingHours"] else "",
            "bedsStaffed": node["bedsStaffed"],
            "bedsNoService": node["bedsNoService"],
            "bedsSelfService": node["bedsSelfService"],
        }
        records.append(record)
    df = pd.DataFrame(records)
    return df


In [5]:
def getCounterTable(df: pd.DataFrame, column_name: str, sort_ascending=False):
	counts = df[column_name].value_counts()

	sorted_counts = counts.sort_values(ascending=sort_ascending)

	return pd.DataFrame({'Value': sorted_counts.index, 'Count': sorted_counts.values})

### Download from UT.no sin API

#### Static data
The API only allows the download of up to 500 huts at the same time. Therefore one needs to trigger the API several times.

At the moment, the number of huts is 646 (2024-03-15), so it will hardly be necessary to do more than 2 times in the future.

However, there is a static value being set to 500 and an automatism checking if there is more data later.

If this somehow does not work, try to change the static number, they might have changed the maximum

In [11]:
request_huts_per_request = 500
folder_path = './data/'

In [12]:
more_data_available = True
counter_pagination = 0
afterCursor = None

while more_data_available:
    post_data = {
        "operationName": "FindCabins",
        "variables": {
            "input": {
                "pageOptions": {
                    "limit": request_huts_per_request,
                    "afterCursor": afterCursor,
                    "orderByDirection": "DESC",
                    "orderBy": "ID",
                },
                "filters": {"and": [{"dntCabin": {"value": True}}]},
            }
        },
        "query": "query FindCabins($input: NTB_FindCabinsInput) {\n  ntb_findCabins(input: $input) {\n    totalCount\n    pageInfo {\n      hasNextPage\n      endCursor\n      __typename\n    }\n    edges {\n      node {\n        ...CabinFragment\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment CabinFragment on NTB_Cabin {\n  id\n  name\n  serviceLevel\n  bedsToday\n  bedsStaffed\n  bedsNoService\n  bedsSelfService\n  bedsWinter\n  dntCabin\n  owner {\n    name\n    __typename\n  }\n  accessibilities {\n    id\n    name\n    __typename\n  }\n  openingHours {\n    allYear\n    from\n    to\n    serviceLevel\n    key\n    __typename\n  }\n  geometry\n  media {\n    id\n    uri\n    type\n    description\n    tags\n    __typename\n  }\n  areas {\n    id\n    name\n    __typename\n  }\n  __typename\n}\n",
    }

    p_data = json.dumps(post_data)
    ua = 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0'

    # Make post request
    dnt_hytter_response = requests.post(
        "https://api.ut.no",
        data=p_data,
        headers={"Content-Type": "application/json", "User-Agent": ua},
    )

    if dnt_hytter_response.headers.get('content-type') == 'application/json':
        response_data = dnt_hytter_response.json()
    else:
        response_data = None

    
    page_info = response_data["data"]["ntb_findCabins"]["pageInfo"]

    json_obj = json.dumps(dnt_hytter_response.json(), indent=4)

    with open(f"{folder_path}hytter_{counter_pagination}.json", "w") as outfile:
        outfile.write(json_obj)

    if page_info["hasNextPage"]:
        counter_pagination += 1
        afterCursor = page_info["endCursor"]
    else:
        more_data_available = False

### Read json files

In [13]:
combined_data = read_json_files(folder_path)

### Convert to pandas

In [14]:
df = convert_to_dataframe(combined_data)

### Check length
Was 646 on 2024-03-15, should be "similar" at any later point

In [15]:
len(combined_data)

0

### Show table

In [89]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,areaName,dntKey,bedsStaffed,bedsNoService,bedsSelfService
0,1033,Gressvasshytta,no-service,True,Hemnes Turistforening,66.048069,14.425802,605,Indre Helgeland,dnt-key,0,27,0
1,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli",unlocked,0,9,0
2,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya",dnt-key,0,8,0
3,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland,dnt-key,0,28,0
4,1083,Damtjønna,emergency shelter,True,Trondhjems Turistforening,62.802600,9.891300,678,Trollheimen,unlocked,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,10483,Angeltjønnhytta,self-service,True,DNT Nord-Trøndelag,63.462446,12.058861,516,Nord-Trøndelag,dnt-key,0,0,8
642,10478,Prestøyhytta,self-service,True,DNT Nord-Trøndelag,63.289152,11.341622,623,Sylan / Bealjehkh,dnt-key,0,0,16
643,10477,Kvitfjellhytta,self-service,True,DNT Nord-Trøndelag,63.343956,11.255825,500,Sylan / Bealjehkh,dnt-key,0,0,11
644,10475,Ramsjøhytta,self-service,True,Trondhjems Turistforening,63.163908,11.722157,771,Sylan / Bealjehkh,dnt-key,0,0,34


### Generate map

In [90]:
m = folium.Map(location=[60.472, 8.468], zoom_start=6)

# Add markers for each point in the DataFrame
for i, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],
        tooltip=f"Height: {row['height']} meters"
    ).add_to(m)

# Display the map
m.save('map.html')  # Save the map to an HTML file

In [91]:
m

### Export to GPX file

In [92]:
gpx = gpxpy.gpx.GPX()

# Create a GPX waypoint for each row in the DataFrame
for i, row in df.iterrows():
    waypoint = gpxpy.gpx.GPXWaypoint(latitude=row['latitude'], longitude=row['longitude'], elevation=row['height'])
    waypoint.name = row['name']
    gpx.waypoints.append(waypoint)

# Save the GPX to a file
with open('points.gpx', 'w') as f:
    f.write(gpx.to_xml())


### Statistics for nerds

In [93]:
df

,id,name,serviceLevel,dntCabin,ownername,latitude,longitude,height,areaName,dntKey,bedsStaffed,bedsNoService,bedsSelfService
0,1033,Gressvasshytta,no-service,True,Hemnes Turistforening,66.048069,14.425802,605,Indre Helgeland,dnt-key,0,27,0
1,1046,Sylvarnes,no-service,True,Vik Turlag,61.093934,6.295336,50,"Stølsheimen, Bergsdalen og Vossafjelli",unlocked,0,9,0
2,1077,Fossestua,no-service,True,DNT Harstad og Omegn,68.634948,16.047413,18,"Lofoten, Vesterålen og Hinnøya",dnt-key,0,8,0
3,1079,Kjensvasshytta,no-service,True,Hemnes Turistforening,66.065423,14.261541,529,Indre Helgeland,dnt-key,0,28,0
4,1083,Damtjønna,emergency shelter,True,Trondhjems Turistforening,62.802600,9.891300,678,Trollheimen,unlocked,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,10483,Angeltjønnhytta,self-service,True,DNT Nord-Trøndelag,63.462446,12.058861,516,Nord-Trøndelag,dnt-key,0,0,8
642,10478,Prestøyhytta,self-service,True,DNT Nord-Trøndelag,63.289152,11.341622,623,Sylan / Bealjehkh,dnt-key,0,0,16
643,10477,Kvitfjellhytta,self-service,True,DNT Nord-Trøndelag,63.343956,11.255825,500,Sylan / Bealjehkh,dnt-key,0,0,11
644,10475,Ramsjøhytta,self-service,True,Trondhjems Turistforening,63.163908,11.722157,771,Sylan / Bealjehkh,dnt-key,0,0,34


##### Service level (betjent, selv-betjent, ubetjent)

In [94]:
getCounterTable(df, "serviceLevel")

,Value,Count
0,no-service,340
1,self-service,195
2,staffed,47
3,emergency shelter,34
4,food service,17
5,no-service (no beds),10
6,closed,3


##### Region

In [95]:
getCounterTable(df, "areaName").head(20)

,Value,Count
0,Oslomarka,41
1,Ryfylke,29
2,Hardangervidda,27
3,"Stølsheimen, Bergsdalen og Vossafjelli",26
4,Breheimen med Jostedalsbreen,24
5,Hedmarken og Hedmarksvidda,22
6,Saltfjellet og Svartisen,18
7,Jotunheimen,18
8,"Lofoten, Vesterålen og Hinnøya",16
9,Trollheimen,16


##### Owner (Is DNT or not)

In [96]:
getCounterTable(df, "dntCabin")

,Value,Count
0,True,646


In [97]:
getCounterTable(df, "ownername")

,Value,Count
0,DNT Oslo og Omegn,143
1,Stavanger Turistforening,46
2,Bergen og Hordaland Turlag,27
3,Kristiansund og Nordmøre Turistforening,26
4,DNT Drammen og Omegn,24
...,...,...
62,Sør-Varanger Turlag,1
63,Odda/Ullensvang Turlag,1
64,Årdal Turlag,1
65,DNT Nedre Glomma,1


##### DNT-Nøkkel neaded

In [98]:
getCounterTable(df, "dntKey")

,Value,Count
0,dnt-key,292
1,unlocked,172
2,special key,89
3,,10
